In [1]:
# Install the transformers library
!pip install transformers
import torch
import numpy as np 
from transformers import AutoTokenizer, AutoModelForSequenceClassification

     |████████████████████████████████| 2.5MB 23.0MB/s 
     |████████████████████████████████| 3.3MB 46.0MB/s 
     |████████████████████████████████| 901kB 49.3MB/s 


In [2]:
# Load tokenizer and model, create trainer
model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [3]:
# sentence to be analyzed
sentence = "What a beautiful day!"

In [4]:
# Tokenize texts and create prediction data set
tokenized_texts = tokenizer(sentence, truncation=True, padding=True, return_tensors="pt")

In [5]:
# Prediction
# tensor ([1]) -> tensor([[1]])
labels = torch.tensor([1]).unsqueeze(0)
outputs = model(**tokenized_texts, labels=labels)
loss, logits = outputs[:2]
# tensor ([[]]) -> tensor ([])
logits = logits.squeeze(0)

In [6]:
# Probabilities
# perform softmax on logits
probabilities_tensor = torch.nn.functional.softmax(logits, dim=0)

# detaches tensor from graph, convert to numpy array
probabilities_numpy = np.array(probabilities_tensor.detach().numpy())

In [7]:
# Obtain emotion probabilities (sorted)
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

# sort np-array by values low-high, flip to high-low
emotion_values = np.argsort(probabilities_numpy)[::-1]

In [8]:
# Print emotion labels with respective prediction values
for emotion in range(len(probabilities_numpy)):
    # current emotion
    em = emotion_labels[emotion_values[emotion]]
    # current probability
    prob = probabilities_numpy[emotion_values[emotion]]
    print(f"{em}:\n" + f"{np.round(float(prob), 10)}\n")

joy:
0.957724154

surprise:
0.0291438121

neutral:
0.0058702491

sadness:
0.0042336052

anger:
0.0013013791

fear:
0.001256481

disgust:
0.0004703144

